# Infer a schema to extract data from files

In this notebook, we will demonstrate how to infer a schema from a set of files and using it to extract structured data from invoice PDF files.

The steps are:
1. Infer a schema from the invoices files.
2. Extract structured data (i.e. JSONs) from invoice PDF files

Additional Resources:
- `LlamaExtract`: https://docs.cloud.llamaindex.ai/

## Setup
Install `llama-extract` client library:

In [ ]:
%pip install llama-extract

Apply `nest_asyncio` and bring your own LlamaCloud API key:

In [ ]:
# llama-extract is async-first, running the sync code in a notebook requires the use of nest_asyncio
import nest_asyncio

nest_asyncio.apply()

import os

os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."

## Infer the schema
First, let's infer the schema using the invoice files with `LlamaExtract`.

In [ ]:
from llama_extract import LlamaExtract

extractor = LlamaExtract()

extraction_schema = extractor.infer_schema(
    "Test Schema", ["./data/noisebridge_receipt.pdf", "./data/parallels_invoice.pdf"]
)

Preview the inferred schema:

In [ ]:
print(extraction_schema.data_schema)

{'type': 'object', 'properties': {'Invoice': {'type': 'object', 'properties': {'total': {'type': 'string'}, 'products': {'type': 'string'}, 'salesTax': {'type': 'string'}, 'subtotal': {'type': 'string'}, 'invoiceDate': {'type': 'string'}, 'invoiceNumber': {'type': 'string'}, 'billingAddress': {'type': 'object', 'properties': {'city': {'type': 'string'}, 'name': {'type': 'string'}, 'country': {'type': 'string'}, 'postalCode': {'type': 'string'}}}, 'paymentDetails': {'type': 'object', 'properties': {'taxId': {'type': 'string'}, 'merchant': {'type': 'string'}, 'merchantAddress': {'type': 'object', 'properties': {'city': {'type': 'string'}, 'suite': {'type': 'string'}, 'street': {'type': 'string'}, 'country': {'type': 'string'}, 'postalCode': {'type': 'string'}}}, 'creditCardLastFour': {'type': 'string'}}}, 'referenceNumber': {'type': 'string'}}}}}


## Extract structured data
Now with the schema, we can extract structured data (i.e. JSON) from the our invoices files.

In [ ]:
extractions = extractor.extract(
    extraction_schema.id,
    ["./data/noisebridge_receipt.pdf", "./data/parallels_invoice.pdf"],
)

Extracting files: 100%|██████████| 2/2 [00:14<00:00,  7.11s/it]


Preview the extracted data:

In [ ]:
print(extractions[0].data)

{'Invoice': {'total': '$119.99', 'products': 'Parallels Desktop for Mac Pro Edition (1 Year)', 'salesTax': '$0.00', 'subtotal': '$119.99', 'invoiceDate': 'Jul 23, 2024', 'invoiceNumber': 'BKD-73649835575', 'billingAddress': {'city': 'California', 'name': 'Laurie Voss', 'country': 'United States', 'postalCode': '94110'}, 'paymentDetails': {'taxId': '20-4503251', 'merchant': 'Cleverbridge, Inc.', 'merchantAddress': {'city': 'Chicago', 'suite': 'Suite 700', 'street': '350 N Clark', 'country': 'United States', 'postalCode': '60654'}, 'creditCardLastFour': '4469'}, 'referenceNumber': '474534804'}}
